In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, TimestampType, DateType, BooleanType
from pyspark.sql import functions as f
from pyspark.sql.functions import isnan, when, count, col, year, lower, regexp_replace, length

In [0]:
spark = SparkSession.builder.appName('Eluvio').getOrCreate()

In [0]:
#load data 
path = '/FileStore/tables/Eluvio_DS_Challenge.csv'
df = spark.read.csv(path, header = 'true')

In [0]:
df.show()

+------------+------------+--------+----------+--------------------+-------+--------------+---------+
time_created|date_created|up_votes|down_votes| title|over_18| author| category|
+------------+------------+--------+----------+--------------------+-------+--------------+---------+
 1201232046| 2008-01-25| 3| 0|Scores killed in ...| False| polar|worldnews|
 1201232075| 2008-01-25| 2| 0|Japan resumes ref...| False| polar|worldnews|
 1201232523| 2008-01-25| 3| 0|US presses Egypt ...| False| polar|worldnews|
 1201233290| 2008-01-25| 1| 0|Jump-start econom...| False| fadi420|worldnews|
 1201274720| 2008-01-25| 4| 0|Council of Europe...| False| mhermans|worldnews|
 1201287889| 2008-01-25| 15| 0|Hay presto! Farme...| False|Armagedonovich|worldnews|
 1201289438| 2008-01-25| 5| 0|Strikes, Protests...| False| Clythos|worldnews|
 1201536662| 2008-01-28| 0| 0|The U.N. Mismanag...| False| Moldavite|worldnews|
 1201558396| 2008-01-28| 4| 0|Nicolas Sarkozy t...| False| Moldavite|worldnews|
 1201635869| 2008-01-29| 3| 0|US plans for miss...| False| JoeyRamone63|worldnews|
 1201701773| 2008-01-30| 4| 0|Archbishop of Can...| False| bgholt1970|worldnews|
 1201719875| 2008-01-30| 9| 0|Top US Envoy: Vio...| False| smacfarl|worldnews|
 1201722210| 2008-01-30| 1| 0|Team building flo...| False| Aerik|worldnews|
 1201765485| 2008-01-31| 1| 0|Migrant workers t...| False| markansoul|worldnews|
 1201794451| 2008-01-31| 2| 0| Sarkozy, Girlfri...| False| Moldavite|worldnews|
 1201794900| 2008-01-31| 3| 0|Nicolas Sarkozy, ...| False| Moldavite|worldnews|
 1201799407| 2008-01-31| 7| 0|Mass Evacuations ...| False| smacfarl|worldnews|
 1201802685| 2008-01-31| 3| 0|Poor Haitians Res...| False|Armagedonovich|worldnews|
 1201834758| 2008-02-01| 0| 0|European Commissi...| False| Moldavite|worldnews|
 1201886520| 2008-02-01| 4| 0|Rambo banned in B...| False| JoeyRamone63|worldnews|
+------------+------------+--------+----------+--------------------+-------+--------------+---------+
only showing top 20 rows

##### check and remove missing value

In [0]:
print((df.count(), len(df.columns)))

(520310, 8)

In [0]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------+------------+--------+----------+-----+-------+------+--------+
time_created|date_created|up_votes|down_votes|title|over_18|author|category|
+------------+------------+--------+----------+-----+-------+------+--------+
 0| 1254| 1587| 6531| 9781| 19944| 20245| 20386|
+------------+------------+--------+----------+-----+-------+------+--------+

In [0]:
data = df.dropna()

In [0]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+------------+------------+--------+----------+-----+-------+------+--------+
time_created|date_created|up_votes|down_votes|title|over_18|author|category|
+------------+------------+--------+----------+-----+-------+------+--------+
 0| 0| 0| 0| 0| 0| 0| 0|
+------------+------------+--------+----------+-----+-------+------+--------+

In [0]:
print((data.count(), len(data.columns)))

(499924, 8)

##### modify data types

In [0]:
# check data type
data.printSchema()

root
-- time_created: string (nullable = true)
-- date_created: string (nullable = true)
-- up_votes: string (nullable = true)
-- down_votes: string (nullable = true)
-- title: string (nullable = true)
-- over_18: string (nullable = true)
-- author: string (nullable = true)
-- category: string (nullable = true)

In [0]:
data = data.withColumn("date_created",data.date_created.cast(DateType()))\
.withColumn('up_votes', data.up_votes.cast(IntegerType()))\
.withColumn('down_votes', data.down_votes.cast(IntegerType()))\
.withColumn('over_18', data.over_18.cast(BooleanType()))

In [0]:
data.printSchema()

root
-- time_created: string (nullable = true)
-- date_created: date (nullable = true)
-- up_votes: integer (nullable = true)
-- down_votes: integer (nullable = true)
-- title: string (nullable = true)
-- over_18: boolean (nullable = true)
-- author: string (nullable = true)
-- category: string (nullable = true)

##### quick look at each variables

In [0]:
data = data.withColumn('year_created', year(data['date_created']))

In [0]:
data['year_created', 'up_votes'].groupBy('year_created').mean('up_votes').show()
# there are significant variation in number of up_votes from 2009 to 2016
# but it can be seperate into three period, 2008-2010, 2011-2013, 2014-2016

+------------+------------------+
year_created| avg(up_votes)|
+------------+------------------+
 null| 81.0|
 2009|25.553955664022777|
 2010| 38.34982764876633|
 2008|15.823764664059723|
 2012| 92.65991861648017|
 2011|53.987826329750106|
 2013| 91.6012132492331|
 2014|119.45837226991199|
 2015| 161.5054481658018|
 2016| 182.0056537879158|
+------------+------------------+

In [0]:
data.groupBy('year_created').count().show()

+------------+-----+
year_created|count|
+------------+-----+
 null| 72|
 2009|24585|
 2010|22048|
 2008|22504|
 2012|39320|
 2011|35815|
 2013|87039|
 2014|92030|
 2015|94619|
 2016|81892|
+------------+-----+

In [0]:
data.groupBy('category').count().show()
# the number of titles belong to worldnews class is far more than others
# so we can took it as only one category

+--------------------+------+
 category| count|
+--------------------+------+
 False| 3|
 Kaffir| 2|
 worldnews|499899|
 Wordie| 1|
 ObamaFraud| 1|
 shocking"| 1|
 galt1776| 1|
 Wikileaks and Hu...| 1|
 and a pond in th...| 1|
 anutensil| 2|
 to bring the rev...| 1|
 for outbreaks of...| 1|
 DrRichardCranium| 9|
 the Netherlands| 1|
+--------------------+------+

In [0]:
data.groupBy('over_18').count().show()

+-------+------+
over_18| count|
+-------+------+
 null| 25|
 true| 315|
 false|499584|
+-------+------+

##### Research Question: Popularity prediction using data from 2014-2016

###### extract data from 2014-2016

In [0]:
df = data.filter(('year_created== 2014') or ('year_created== 2015') or ('year_created== 2016')).select('up_votes','title')

###### label data with two classes: popular, unpopular

In [0]:
# take median as threshold
threshold = df.approxQuantile('up_votes', [0.5], 0.25)

In [0]:
threshold
# half of the videos cannot get at least 2 up_votes

Out[21]: [2.0]

In [0]:
# add class to the datafrom
df = df.withColumn('class', f.when((df['up_votes']>threshold[0]),'popular').otherwise('unpopular'))

In [0]:
df.show()

+--------+--------------------+---------+
up_votes| title| class|
+--------+--------------------+---------+
 0|Syria: UN humanit...|unpopular|
 0|Homeless recruite...|unpopular|
 18|Pollution Rising,...| popular|
 22|Chinese recycling...| popular|
 0|20th century Fox ...|unpopular|
 0|Al Jazeera team f...|unpopular|
 13|Lebanon says army...| popular|
 17|Antarctica Explor...| popular|
 7|Former Internatio...| popular|
 4|South Sudan s gov...| popular|
 4|South Sudan rebel...| popular|
 23|The Cancer Divide...| popular|
 10|Global warming co...| popular|
 0|Pope Francis has ...|unpopular|
 423|Russia wishes NAT...| popular|
 11|Don t be blind to...| popular|
 1161|WikiLeaks’ Julian...| popular|
 1|North Korean lead...|unpopular|
 0| Regards to John ...|unpopular|
 0|Bangladesh factor...|unpopular|
+--------+--------------------+---------+
only showing top 20 rows

###### text data clean

In [0]:
# to lowercase
df = df.withColumn('text', lower(df['title']))

In [0]:
# remove numbers and punctuations
df = df.withColumn('text_only', regexp_replace(df['text'],'\W', ' '))

In [0]:
# check text length (see if there is clear difference in length of two classes)
df = df.withColumn('length', length(df['text_only']))
df.groupBy('class').mean('length').show()
# length difference can be ignored 

+---------+-----------------+
 class| avg(length)|
+---------+-----------------+
 popular| 92.962459684681|
unpopular|79.89017415748489|
+---------+-----------------+

###### create data processing pipeline

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [0]:
tokenizer = Tokenizer(inputCol = 'text_only', outputCol = 'token_text')
stop_remover = StopWordsRemover(inputCol = 'token_text', outputCol = 'stop_tokens')
hashing_tf = HashingTF(inputCol = 'stop_tokens', outputCol = 'rawFeatures')
idf = IDF(inputCol = 'rawFeatures', outputCol = 'tf_idf')
class_to_numeric = StringIndexer(inputCol = 'class', outputCol = 'Label')

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols = ['tf_idf'], outputCol = 'features')

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages = [class_to_numeric, tokenizer, stop_remover, hashing_tf, idf, clean_up])

In [0]:
cleaner = data_prep_pipe.fit(df)

In [0]:
clean_data = cleaner.transform(df)

In [0]:
clean_data = clean_data.select('features','Label')

###### modeling and predition

In [0]:
# split data into training and test set
training, test = clean_data.randomSplit([0.7,0.3])

In [0]:
# select model
from pyspark.ml.classification import (NaiveBayes, LogisticRegression, 
                                       DecisionTreeClassifier, RandomForestClassifier, GBTClassifier)

In [0]:
nb = NaiveBayes(labelCol = 'Label',featuresCol = 'features')
rf = RandomForestClassifier(labelCol = 'Label',featuresCol = 'features')
# gbt = GBTClassifier(maxIter = 10, labelCol = 'Label')

In [0]:
nb_model = nb.fit(training)
nb_predictions = nb_model.transform(test)

In [0]:
rf_model = rf.fit(training)
rf_predictions = rf_model.transform(test)

In [0]:
# gbt_model = gbt.fit(training)
# gbt_predictions = gbt_model.transform(test)

###### evaluation

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol = 'Label', predictionCol = 'prediction', 
                                              metricName = 'accuracy')

In [0]:
nb_accuracy = evaluator.evaluate(nb_predictions)

In [0]:
print("Accuracy of NaiveBayes Classifier is = %g"% (nb_accuracy))

Accuracy of NaiveBayes Classifier is = 0.627071

In [0]:
rf_accuracy = evaluator.evaluate(rf_predictions)
print("Accuracy of Random Forest Classifier is = %g"% (rf_accuracy))

Accuracy of Random Forest Classifier is = 0.667474